In [3]:
from roboflow import Roboflow
rf = Roboflow(api_key="bwISTK3XdVI7LVTIYM62")
project = rf.workspace("zwdyolov5").project("road-segmenttation")
dataset = project.version(4).download("png-mask-semantic")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to road-segmenttation-4 in png-mask-semantic:: 100%|██████████| 352/352 [00:00<00:00, 3387.98it/s]


In [2]:
import time
import copy
import torch
from torch import optim, nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision.models import resnet18
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import sys
sys.path.append("..")
from IPython import display
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore") # 忽略警告

In [6]:
from pathlib import Path
import os
import cv2
def load_image(type):
    root = f"./road-segmenttation-4/{type}"
    image_list = []
    for root, dirs, files in os.walk(root):
        for file in files:
            file_path = os.path.join(root, file)
            if file_path[-3:] == "png":
                frame = cv2.imread(file_path).
                
    

In [7]:
load_image()

./road-segmenttation-4/train/IMG_6091_jpg.rf.4feaabe6b07bf3ff5d3dded23d226cf5.jpg
./road-segmenttation-4/train/IMG_0067_jpg.rf.dec49af146facf44d01c1e3edf220c18.jpg
./road-segmenttation-4/train/IMG_6100_jpg.rf.1b2c6e49fd111d507b8d3acee97a68f9_mask.png
./road-segmenttation-4/train/IMG_0069_jpg.rf.3692e4a5b7622cef7114830a4e1917bb.jpg
./road-segmenttation-4/train/IMG_0067_jpg.rf.5e772857c1cb642abec4bc7b734cb597.jpg
./road-segmenttation-4/train/IMG_6096-1-_jpg.rf.df14c1dd228b6069259816552ec83a80_mask.png
./road-segmenttation-4/train/IMG_0060_jpg.rf.0ec8d843ea188e39147da96bbd72d0f8.jpg
./road-segmenttation-4/train/IMG_6094_jpg.rf.8be4139766a48f778278e28bfb6a247c_mask.png
./road-segmenttation-4/train/IMG_0051_jpg.rf.74769bca3547700c161ef2488c416542_mask.png
./road-segmenttation-4/train/IMG_6095_jpg.rf.f54ac826170b9f3e65b552aeb8a5b903_mask.png
./road-segmenttation-4/train/IMG_6101_jpg.rf.1f7ca46d2b6e5c9d356b12cb56fa9bea.jpg
./road-segmenttation-4/train/IMG_0079_jpg.rf.b789ba855337ccc8ee7bf18c9